In [3]:
# Dataloader ： utils/loder.pyに記述するべき内容
import numpy as np
import cv2
from math import ceil
from scipy import ndimage
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras import utils
import keras
from tensorflow.keras.datasets.mnist import load_data
from keras.preprocessing.image import ImageDataGenerator
import os
import glob
import tensorflow as tf
from PIL import Image
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
%matplotlib inline

In [85]:
# 完成
def xml_to_labels(annotation_file):
    '''
    指定したxmlファイル(JSON)から、画像ファイル名とアノテーションを取得する
    bboxはJSONの第1層のもののみ取得
    （例：object=personの中に hand, foot...等が含まれるデータもある）
    
    Parameters
    --------------
    annotations_file : str (or pathlib.PosixPath)
        xmlファイルパス

    Returns
    --------------
    img_filename : str
        画像ファイル名
    df_annotations : pandas.DataFrame, shape=(num_object, 5)
        画像に写っている物体(num_object個)に対し、下記の5つのカラムを持つDF
        ・xmin: BBOXのx座標のmin
        ・ymin: BBOXのy座標のmin
        ・xmax: BBOXのx座標のmax
        ・ymax: BBOXのy座標のmax
        ・label: クラスラベル
    '''
    with open(annotation_file) as f:
        xml_data = f.read()  # xmlファイルの内容を読み込む

    # xml操作
    root = ET.XML(xml_data)
    obj_to_int = lambda x: int(x.text)
    df_annotations = pd.DataFrame(columns=['xmin', 'ymin', 'xmax', 'ymax', 'width', 'height', 'name', "pose", "truncated", "difficult"])
    for i, child in enumerate(root):
        if child.tag == 'folder':
            folder = child.text   
        if child.tag == 'filename':
            filename = child.text        
        if child.tag == 'path':
            path = child.text                   
        if child.tag == 'segmented':
            segmented = child.text
            
        if child.tag == 'source':
            for subchild in child:
                if subchild.tag == "database":
                    database = subchild.text
                    
        if child.tag == 'size':
            for subchild in child:
                if subchild.tag == 'width':
                    width = int(subchild.text)
                if subchild.tag == 'height':
                    height = int(subchild.text)
                if subchild.tag == 'depth':
                    depth = int(subchild.text)
                    
                    
        if child.tag == 'object':
            # 各objectにname,bndboxタグが必ず1つのみついていることを想定して値を読み取る
            for subchild in child:
                if subchild.tag == 'pose':
                    pose = subchild.text
                if subchild.tag == 'truncated':
                    truncated = subchild.text
                if subchild.tag == 'difficult':
                    difficult = subchild.text
                if subchild.tag == 'name':
                    label = subchild.text
                if subchild.tag == 'bndbox':
                    xmin, ymin, xmax, ymax = tuple(map(obj_to_int, subchild.getchildren()))
            # BBOXの幅と高さを計算
            #w = xmax - xmin
            #h = ymax - ymin
            # DFに追加
            df_annotations = df_annotations.append(
                {"folder" : folder, "filename" : filename, "path" : path, "database" : database, "segmented": segmented, 'xmin': xmin, 'ymin': ymin, 'xmax': xmax, 'ymax': ymax, 'width': width, "depth" : depth, 'height': height, 'name': label, "pose" : pose, "truncated" : truncated, "difficult" : difficult},
                ignore_index=True
            )
    return df_annotations

In [86]:
# 完成
def csv_to_labels(annotation_file):
    df_annotations = pd.read_csv(annotation_file)
    img_filename = os.path.splitext(annotation_file.split("/")[-1])[0]
    return img_filename, df_annotations

In [87]:
# 完成
def get_trimed_img(img, ws, hs, trimed_img_shape):
    trimed_img = {}
    zettai_pos = {}
    
    cnt = 0

    for h in range(hs):
        for w in range(ws):
            x_start = trimed_img_shape[1] * w
            y_start = trimed_img_shape[0] * h
            x_end = x_start+trimed_img_shape[1]
            y_end = y_start+trimed_img_shape[0]

            if x_start+trimed_img_shape[1] >= img.shape[1]:
                x_start = img.shape[1] - trimed_img_shape[1]
                x_end = img.shape[1]

            if y_start+trimed_img_shape[0] >= img.shape[0]:
                y_start = img.shape[0] - trimed_img_shape[0]
                y_end = img.shape[0]
     
            trimed_img[cnt] = img[y_start:y_end,x_start:x_end]           
            zettai_pos[cnt] = {"x_start": x_start, "x_end": x_end, "y_start": y_start, "y_end": y_end}
            cnt = cnt + 1
    return trimed_img, zettai_pos

In [88]:
# 完成

def get_trimed_bbox(df_annotations, zettai_pos, trimed_img_shape):
    trimed_bbox = {}
    for i in range(len(zettai_pos.keys())):
    #     print("x_start : ", zettai_pos[i]["x_start"])
    #     print("x_end : ", zettai_pos[i]["x_end"])
    #     print("y_start : ", zettai_pos[i]["y_start"])
    #     print("y_end : ", zettai_pos[i]["y_end"])
        df_x_trimed = df_annotations[(df_annotations["xmax"] >= zettai_pos[i]["x_start"]) & (df_annotations["xmin"] <= zettai_pos[i]["x_end"])]
        df_x_trimed.loc[df_x_trimed["xmin"] <= zettai_pos[i]["x_start"], ["xmin"]] = zettai_pos[i]["x_start"]
        df_x_trimed.loc[df_x_trimed["xmax"] >= zettai_pos[i]["x_end"], ["xmax"]] = zettai_pos[i]["x_end"]

        df_x_y_trimed = df_x_trimed[(df_x_trimed["ymax"] >= zettai_pos[i]["y_start"]) & (df_x_trimed["ymin"] <= zettai_pos[i]["y_end"])]
        df_x_y_trimed.loc[df_x_y_trimed["ymin"] <= zettai_pos[i]["y_start"], ["ymin"]] = zettai_pos[i]["y_start"]
        df_x_y_trimed.loc[df_x_y_trimed["ymax"] >= zettai_pos[i]["y_end"], ["ymax"]] = zettai_pos[i]["y_end"]
              
        # todo broadcast演算で出てくるwarning消す
        df_x_y_trimed[["xmin", "xmax"]] = df_x_y_trimed[["xmin", "xmax"]] - zettai_pos[i]["x_start"] 
        df_x_y_trimed[["ymin", "ymax"]] = df_x_y_trimed[["ymin", "ymax"]] - zettai_pos[i]["y_start"]
        df_x_y_trimed['width'] = trimed_img_shape[1]
        df_x_y_trimed['height'] = trimed_img_shape[0]
        trimed_bbox[i] = df_x_y_trimed
        
    return trimed_bbox

In [89]:
# 完成
def run_trim(img, df_annotations, trimed_img_shape):
    ws = img.shape[1] // trimed_img_shape[1] + 1
    hs = img.shape[0] // trimed_img_shape[0] + 1
    print("横方向 : ", ws)
    print("縦方向 : ", hs)
    print("{} images will be created".format(ws * hs))
    
    trimed_img, zettai_pos = get_trimed_img(img, ws, hs, trimed_img_shape)
    trimed_bbox = get_trimed_bbox(df_annotations, zettai_pos, trimed_img_shape)
    return trimed_img, trimed_bbox

In [222]:
test = "a3bdae-24cf-4162-97b1-3b9f0a01a4e9-11.png"

In [224]:
def add_one_object(roots, row):
    object_ = ET.SubElement(roots, 'object')
    name = ET.SubElement(object_, 'name')
    name.text = row["name"]
    pose = ET.SubElement(object_, 'pose')
    pose.text = row["pose"]
    truncated = ET.SubElement(object_, 'truncated')
    truncated.text = row["truncated"]
    difficult = ET.SubElement(object_, 'difficult')
    difficult.text = row["difficult"]

    bndbox = ET.SubElement(object_, 'bndbox')
    xmin = ET.SubElement(bndbox, 'xmin')
    xmin.text = str(row["xmin"])
    ymin = ET.SubElement(bndbox, 'ymin')
    ymin.text = str(row["ymin"])
    xmax = ET.SubElement(bndbox, 'xmax')
    xmax.text = str(row["xmax"])
    ymax = ET.SubElement(bndbox, 'ymax')
    ymax.text = str(row["ymax"])
    return roots

def make_xml(dc, trim_id):
    roots = ET.Element('annotation')
    folder = ET.SubElement(roots, 'folder')
    folder.text = dc["folder"].iloc[0]
    filename = ET.SubElement(roots, 'filename')
    filename.text = "{0}_{1}.{2}".format(dc["filename"].iloc[0].split(".")[0], trim_id, dc["filename"].iloc[0].split(".")[-1])
    path = ET.SubElement(roots, 'path')
    path.text = dc["path"].iloc[0]
    source = ET.SubElement(roots, 'source')
    database = ET.SubElement(source, "database") 
    database.text = dc["database"].iloc[0]
    
    size = ET.SubElement(roots, 'size')
    width = ET.SubElement(size, "width") 
    width.text = str(dc["width"].iloc[0])
    height = ET.SubElement(size, "height") 
    height.text = str(dc["height"].iloc[0])
    depth = ET.SubElement(size, "depth") 
    depth.text = str(dc["depth"].iloc[0])

    segmented = ET.SubElement(roots, 'segmented')
    segmented.text = dc["segmented"].iloc[0]
    
    # 以下をforループで回したい
    for index, row in dc.iterrows():
        roots = add_one_object(roots, row)
     
    #XML文字列の出力
    ET.dump(roots)

    #XMLファイルの生成
    tree = ET.ElementTree(roots) 

    # 生成したXMLファイル名を返す
    return tree

In [225]:
# todo 関数化・変数名揃える
def main(original_data_path, output_data_path):
    image_filenames = glob.glob(original_data_path + "/*.png")
    annotation_filenames = glob.glob(original_data_path + "/*.xml")
    assert len(image_filenames) == len(annotation_filenames),"画像とアノテーションファイルの数が違います"

    num = len(image_filenames)   
    trimed_imgs = {}
    trimed_bboxes = {}

    for i in range(num):
        # annotationファイル名を基準に画像を読み込む
        df_annotations = xml_to_labels(annotation_filenames[i])
        img = cv2.imread(original_data_path + df_annotations["filename"][0])

        # triming 実行
        trimed_imgs["image {}".format(i)], trimed_bboxes["image {}".format(i)] = run_trim(img, df_annotations, trimed_img_shape = (650,650)) # todo 繋ぎこみ

    for i, image_id in enumerate(trimed_imgs):
        for trim_id in trimed_imgs[image_id]:
            # 画像ファイルの保存
            filename = "{0}_{1}".format(os.path.splitext(annotation_filenames[i].split("/")[-1])[0], trim_id)
            cv2.imwrite(output_data_path + "{}.png".format(filename), trimed_imgs[image_id][trim_id])
            
            
            # xmlではなくcsvに保存したい時
            # trimed_bboxes[image_id][trim_id].to_csv(output_data_path + "{}.csv".format(filename))
            
            # annotationが存在している時はxmlファイルを保存する
            df = trimed_bboxes[image_id][trim_id]           
            if len(df) != 0:
                print(len(df))
                tree = make_xml(df, trim_id)
                tree.write(output_data_path + '{}.xml'.format(filename))

In [226]:
original_data_path = '/home/esaka/notebooks/data2/train/'
output_data_path = '/home/esaka/notebooks/data2/train/trimed/'
main(original_data_path, output_data_path)

# アノテーションが無い画像はここで削除
xml_filenames = glob.glob(output_data_path + "*.xml")
png_filenames = glob.glob(output_data_path + "*.png")
for png_f in png_filenames:
    if any([os.path.splitext(png_f.split("/")[-1])[0]  == os.path.splitext(xml_f.split("/")[-1])[0] for xml_f in xml_filenames]):
         pass
    else:
        !rm {png_f}
        
xml_filenames = glob.glob(output_data_path + "*.xml")
png_filenames = glob.glob(output_data_path + "*.png")
print(len(png_filenames))
print(len(xml_filenames))

横方向 :  11
縦方向 :  8
88 images will be created


/home/esaka/.local/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/esaka/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


横方向 :  11
縦方向 :  8
88 images will be created
横方向 :  11
縦方向 :  8
88 images will be created
2
<annotation><folder>Downloads</folder><filename>70a3bdae-24cf-4162-97b1-3b9f0a01a4e9-11_0.png</filename><path>C:\Users\9046692\Downloads\70a3bdae-24cf-4162-97b1-3b9f0a01a4e9-11.png</path><source><database>Unknown</database></source><size><width>650</width><height>650</height><depth>1.0</depth></size><segmented>0</segmented><object><name>number</name><pose>Unspecified</pose><truncated>0</truncated><difficult>0</difficult><bndbox><xmin>553</xmin><ymin>537</ymin><xmax>650</xmax><ymax>650</ymax></bndbox></object><object><name>number</name><pose>Unspecified</pose><truncated>0</truncated><difficult>0</difficult><bndbox><xmin>536</xmin><ymin>439</ymin><xmax>650</xmax><ymax>481</ymax></bndbox></object></annotation>
15
<annotation><folder>Downloads</folder><filename>70a3bdae-24cf-4162-97b1-3b9f0a01a4e9-11_1.png</filename><path>C:\Users\9046692\Downloads\70a3bdae-24cf-4162-97b1-3b9f0a01a4e9-11.png</path><

# 画像が正しくトリミング出来ているか確認

In [ ]:
plt.figure(figsize = (20,5))

for i in range(22):
    plt.subplot(2,11,i+1)
    plt.imshow(trimed_img[i])

# todo bboxが正しく抜けているか確認

In [ ]:
plt.figure(figsize = (20,10))
plt.imshow(img)